In [ ]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix


# Load the data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Split the data into features and labels
X_train = train_df.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
y_train = train_df['Survived']
X_test = test_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# Preprocess the data  EDA
X_train['Age'] = X_train['Age'].fillna(X_train['Age'].median())
X_train['Fare'] = X_train['Fare'].fillna(X_train['Fare'].median())
X_train['Embarked'] = X_train['Embarked'].fillna(X_train['Embarked'].mode()[0])
X_train['Sex'] = X_train['Sex'].map({'female': 0, 'male': 1})
X_train['Embarked'] = X_train['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

X_test['Age'] = X_test['Age'].fillna(X_test['Age'].median())
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].median())
X_test['Embarked'] = X_test['Embarked'].fillna(X_test['Embarked'].mode()[0])
X_test['Sex'] = X_test['Sex'].map({'female': 0, 'male': 1})
X_test['Embarked'] = X_test['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})


In [ ]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Split the training data into training and validation sets
X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# Train the KNN model using cross-validation
k_range = range(1, 21)
cv_scores = []
for k in k_range:
    model = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
    cv_scores.append(np.mean(scores))

# Select the best k value based on the highest cross-validation score
best_k = np.argmax(cv_scores) + 1
print('Best k:', best_k)

# Train the KNN model on the training set using the best k value
model = KNeighborsClassifier(n_neighbors=best_k, metric='euclidean')
model.fit(X_train_split, y_train_split)

# Evaluate the model on the validation set
y_pred_valid = model.predict(X_valid_split)
valid_acc = accuracy_score(y_valid_split, y_pred_valid)
valid_precision = precision_score(y_valid_split, y_pred_valid)
valid_recall = recall_score(y_valid_split, y_pred_valid)
valid_f1 = f1_score(y_valid_split, y_pred_valid)
# print('Validation accuracy:', valid_acc)
# print('Validation precision:', valid_precision)
# print('Validation recall:', valid_recall)
# print('Validation F1-score:', valid_f1)

# Apply the trained model to the test set for predictions
y_pred = model.predict(X_test_scaled)
test_df['Survived'] = y_pred


#print(test_df['Survived'])

from sklearn.metrics import confusion_matrix

# calculate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# print the confusion matrix
print('Confusion Matrix:\n', conf_matrix)

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Split the training data into training and validation sets
X_train_split, X_valid_split, y_train_split, y_valid_split = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# Train the Naive Bayes model using cross-validation
model = GaussianNB()
scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
print('Cross-validation scores:', scores)
print('Mean cross-validation score:', np.mean(scores))




from sklearn.metrics import confusion_matrix

# Train the Naive Bayes model on the training set
model.fit(X_train_scaled, y_train)

# Evaluate the model on the validation set
y_pred_valid = model.predict(X_valid_split)
valid_acc = accuracy_score(y_valid_split, y_pred_valid)
print('Validation accuracy:', valid_acc)

# Calculate the confusion matrix for the validation set predictions
conf_matrix_valid = confusion_matrix(y_valid_split, y_pred_valid)
print('Confusion Matrix (Validation Set):\n', conf_matrix_valid)

# Apply the trained model to the test set for predictions
y_pred_test = model.predict(X_test_scaled)
test_df['Survived'] = y_pred_test

# Calculate the confusion matrix for the test set predictions
# Replace y_test with the actual labels for the test set
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
print('Confusion Matrix (Test Set):\n', conf_matrix_test)

In [ ]:
from sklearn.metrics import confusion_matrix

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create an MLPClassifier model
mlp = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=1000)

# Train the model
mlp.fit(X_train_scaled, y_train)

# Evaluate the model on the training set
y_train_pred = mlp.predict(X_train_scaled)
train_acc = accuracy_score(y_train, y_train_pred)
print("Training Accuracy:", train_acc)

# Apply the trained model to the testing set for predictions
y_test_pred = mlp.predict(X_test_scaled)

# Print the predicted values
print("Predicted values for test data:")
print(y_test_pred)

# Calculate the confusion matrix for the training set predictions
conf_matrix_train = confusion_matrix(y_train, y_train_pred)
print('Confusion Matrix (Training Set):\n', conf_matrix_train)

# Calculate the confusion matrix for the test set predictions
# Replace y_test with the actual labels for the test set
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
print('Confusion Matrix (Test Set):\n', conf_matrix_test)

# Comparative Analysis


Compare  the  performance  of the  three  algorithms  (KNN,  Naive  Bayes, and ANN) based on the evaluation metrics  obtained: 

To compare the performance of the three algorithms (KNN, Naive Bayes, and ANN), we can compare their evaluation metrics. Here's a summary of the evaluation metrics for each algorithm, based on the code you provided:

KNN:

Validation accuracy: 0.804
Confusion matrix (validation set):

[[239   8]
 [ 38 133]]



Naive Bayes:

Validation accuracy: 0.789
Confusion matrix (validation set):
 [[84 21]
 [20 54]]

ANN:
Training accuracy: 0.843
Confusion matrix (training set):
 [[529  20]
 [ 73 269]]


Based on these metrics, we can see that the KNN model has the highest validation accuracy, followed  by the Naive Bayes model. However, the ANN model has a higher training accuracy, indicating that it may have more potential to improve with further  optimization.


In terms of strengths and weaknesses, the KNN algorithm is simple and easy to understand, and can work well with non-linear and complex data. However, it may not perform well with high-dimensional data and can be sensitive to outliers. The Naive Bayes algorithm is also simple and fast, and can handle high-dimensional data well. However, it assumes that features are independent and may not work well with correlated features. The ANN algorithm is highly flexible and can learn complex patterns in the data, but it requires more data and computational resources to train, and can be prone to overfitting.


Overall, based on the evaluation metrics and the strengths and weaknesses of each algorithm, it seems that the KNN and Naive Bayes algorithms perform similarly well on the Titanic dataset, while the ANN algorithm may have more potential for improvement with further optimization.





